In [17]:
!pip install annoy

     |████████████████████████████████| 655kB 5.7MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp36-cp36m-linux_x86_64.whl size=390355 sha256=816131517405ca9c95298d4182957727122f5c9a0fa53e4909f3b8a97017b4c5
  Stored in directory: /root/.cache/pip/wheels/3a/c5/59/cce7e67b52c8e987389e53f917b6bb2a9d904a03246fadcb1e
Successfully built annoy


In [18]:
import tensorflow as tf
import tensorflow_hub as hub

import numpy as np
import time

import glob        #for reading file names in a folder
import os.path        
from google.colab import drive
from annoy import AnnoyIndex

In [3]:
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Colab Notebooks/input/') #Add your path of images here

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
pwd

'/content/drive/My Drive/Colab Notebooks/input'

In [12]:
path = os.path.expanduser('/content/drive/My Drive/Colab Notebooks/input/test/')
print(path)

/content/drive/My Drive/Colab Notebooks/input/test/


In [5]:
#function to take input image
def input_img(path):
  
  image = tf.io.read_file(path)  #reads the input image file and returns the datatype of string
  image = tf.io.decode_jpeg(image, channels=3)      # Decodes the image to W x H x 3 shape tensor with type of uint8
  image = tf.image.resize_with_pad(image, 224, 224)    # Resize the image to 224 x 244 x 3 shape tensor

  image  = tf.image.convert_image_dtype(image, tf.float32)[tf.newaxis, ...]

  return image
    





In [14]:
def get_image_feature_vectors():

  i=0
  start_time = time.time()
    
  print("---------------------------------")
  print("Step.1 of 2 - mobilenet_v2_140_224 - Loading Started at %s" %time.ctime())
  print("---------------------------------")

  # Definition of module with using tfhub.dev handle
  module_handle = "https://tfhub.dev/google/imagenet/mobilenet_v2_140_224/feature_vector/4" 
  
  # Load the module
  module = hub.load(module_handle)

  print("---------------------------------")
  print("Step.1 of 2 - mobilenet_v2_140_224 - Loading Completed at %s" %time.ctime())
  print("--- %.2f minutes passed ---------" % ((time.time() - start_time)/60))

  print("---------------------------------")
  print("Step.2 of 2 - Generating Feature Vectors -  Started at %s" %time.ctime())
  #for filename in glob.glob('/Users/erdemisbilen/Angular/fashionWebScraping/images_scraped/full/test/*.jpg'): #assuming gif
  for filename in glob.glob('*.jpg'): #assuming gif
    i = i + 1

    print("-----------------------------------------------------------------------------------------")
    print("Image count                     :%s" %i)
    print("Image in process is             :%s" %filename)

    # Loads and pre-process the image
    img = input_img(filename)

    # Calculate the image feature vector of the img
    features = module(img)   
  
    # Remove single-dimensional entries from the 'features' array
    feature_set = np.squeeze(features)  

    # Saves the image feature vectors into a file for later use

    outfile_name = os.path.basename(filename).split('.')[0] + ".npz"
    #out_path = os.path.join('/Users/erdemisbilen/Angular/fashionWebScraping/images_scraped/feature-vectors/test/', outfile_name)
    out_path = os.path.join(path, outfile_name)
    # Saves the 'feature_set' to a text file
    np.savetxt(out_path, feature_set, delimiter=',')

    print("Image feature vector saved to   :%s" %out_path)
  
  print("---------------------------------")
  print ("Step.2 of 2 - Generating Feature Vectors - Completed at %s" %time.ctime())
  print("--- %.2f minutes passed ---------" % ((time.time() - start_time)/60))
  print("--- %s images processed ---------" %i)
    
get_image_feature_vectors()


---------------------------------
Step.1 of 2 - mobilenet_v2_140_224 - Loading Started at Sun Dec 13 19:44:36 2020
---------------------------------
---------------------------------
Step.1 of 2 - mobilenet_v2_140_224 - Loading Completed at Sun Dec 13 19:45:41 2020
--- 1.07 minutes passed ---------
---------------------------------
Step.2 of 2 - Generating Feature Vectors -  Started at Sun Dec 13 19:45:41 2020
-----------------------------------------------------------------------------------------
Image count                     :1
Image in process is             :dd_tree.jpg
Image feature vector saved to   :/content/drive/My Drive/Colab Notebooks/input/test/dd_tree.npz
-----------------------------------------------------------------------------------------
Image count                     :2
Image in process is             :snake.jpg
Image feature vector saved to   :/content/drive/My Drive/Colab Notebooks/input/test/snake.npz
---------------------------------
Step.2 of 2 - Generating

In [19]:
# Numpy for loading image feature vectors from file
import numpy as np

# Time for measuring the process time
import time

# Glob for reading file names in a folder
import glob
import os.path

# json for storing data in json file
import json

# Annoy and Scipy for similarity calculation
from annoy import AnnoyIndex
from scipy import spatial
#################################################

#################################################
# This function reads from 'image_data.json' file
# Looks for a specific 'filename' value
# Returns the product id when product image names are matched 
# So it is used to find product id based on the product image name
#################################################
def match_id(filename):
  #with open('/Users/erdemisbilen/Angular/fashionWebScraping/jsonFiles/image_data.json') as json_file:
  with open('/image_data.json') as json_file:
    
    for file in json_file:
        seen = json.loads(file)

        for line in seen:
          
          if filename==line['imageName']:
            print(line)
            return line['productId']
            break
#################################################

#################################################
# This function; 
# Reads all image feature vectores stored in /feature-vectors/*.npz
# Adds them all in Annoy Index
# Builds ANNOY index
# Calculates the nearest neighbors and image similarity metrics
# Stores image similarity scores with productID in a json file
#################################################
def cluster():

  start_time = time.time()
  
  print("---------------------------------")
  print ("Step.1 - ANNOY index generation - Started at %s" %time.ctime())
  print("---------------------------------")

  # Defining data structures as empty dict
  file_index_to_file_name = {}
  file_index_to_file_vector = {}
  file_index_to_product_id = {}

  # Configuring annoy parameters
  dims = 1792
  n_nearest_neighbors = 20
  trees = 10000
  
  tmp_path = path + '/test/*.npz'
  # Reads all file names which stores feature vectors 
  allfiles = glob.glob(tmp_path)
  
  t = AnnoyIndex(dims, metric='angular')

  for file_index, i in enumerate(allfiles):
    
  
    # Reads feature vectors and assigns them into the file_vector 
    file_vector = np.loadtxt(i)

    # Assigns file_name, feature_vectors and corresponding product_id
    file_name = os.path.basename(i).split('.')[0]
    file_index_to_file_name[file_index] = file_name
    file_index_to_file_vector[file_index] = file_vector
    file_index_to_product_id[file_index] = match_id(file_name)

    # Adds image feature vectors into annoy index   
    t.add_item(file_index, file_vector)

    print("---------------------------------")
    print("Annoy index     : %s" %file_index)
    print("Image file name : %s" %file_name)
    print("Product id      : %s" %file_index_to_product_id[file_index])
    print("--- %.2f minutes passed ---------" % ((time.time() - start_time)/60))


  # Builds annoy index
  t.build(trees)

  print ("Step.1 - ANNOY index generation - Finished")
  print ("Step.2 - Similarity score calculation - Started ") 
  
  named_nearest_neighbors = []

  # Loops through all indexed items
  for i in file_index_to_file_name.keys():

    # Assigns master file_name, image feature vectors and product id values
    master_file_name = file_index_to_file_name[i]
    master_vector = file_index_to_file_vector[i]
    master_product_id = file_index_to_product_id[i]

    # Calculates the nearest neighbors of the master item
    nearest_neighbors = t.get_nns_by_item(i, n_nearest_neighbors)

    # Loops through the nearest neighbors of the master item
    for j in nearest_neighbors:

      print(j)

      # Assigns file_name, image feature vectors and product id values of the similar item
      neighbor_file_name = file_index_to_file_name[j]
      neighbor_file_vector = file_index_to_file_vector[j]
      neighbor_product_id = file_index_to_product_id[j]

      # Calculates the similarity score of the similar item
      similarity = 1 - spatial.distance.cosine(master_vector, neighbor_file_vector)
      rounded_similarity = int((similarity * 10000)) / 10000.0

      # Appends master product id with the similarity score 
      # and the product id of the similar items
      named_nearest_neighbors.append({
        'similarity': rounded_similarity,
        'master_pi': master_product_id,
        'similar_pi': neighbor_product_id})

    print("---------------------------------") 
    print("Similarity index       : %s" %i)
    print("Master Image file name : %s" %file_index_to_file_name[i]) 
    print("Nearest Neighbors.     : %s" %nearest_neighbors) 
    print("--- %.2f minutes passed ---------" % ((time.time() - start_time)/60))

  
  print ("Step.2 - Similarity score calculation - Finished ") 

  # Writes the 'named_nearest_neighbors' to a json file
  with open('nearest_neighbors.json', 'w') as out:
    json.dump(named_nearest_neighbors, out)

  print ("Step.3 - Data stored in 'nearest_neighbors.json' file ") 
  print("--- Prosess completed in %.2f minutes ---------" % ((time.time() - start_time)/60))

cluster()


---------------------------------
Step.1 - ANNOY index generation - Started at Sun Dec 13 19:53:01 2020
---------------------------------
Step.1 - ANNOY index generation - Finished
Step.2 - Similarity score calculation - Started 
Step.2 - Similarity score calculation - Finished 
Step.3 - Data stored in 'nearest_neighbors.json' file 
--- Prosess completed in 0.01 minutes ---------
